In [1]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


2.8.0+cu128
True
Quadro RTX 3000


In [3]:
from ultralytics import YOLO

# Load models on CPU only to verify file access
model = YOLO("/home/bluro/Vehicle-Distance-Measurement-System/yolo12x.pt").to('cpu')
license_plate_model = YOLO("/home/bluro/Vehicle-Distance-Measurement-System/license-plate.pt").to('cpu')

print("✅ Both YOLO models loaded successfully on CPU.")

✅ Both YOLO models loaded successfully on CPU.


In [4]:
import torch
from ultralytics import YOLO

device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = YOLO("/home/bluro/Vehicle-Distance-Measurement-System/yolo12x.pt").to(device)
license_plate_model = YOLO("/home/bluro/Vehicle-Distance-Measurement-System/license-plate.pt").to(device)

print(f"✅ Models loaded successfully on {device.upper()}!")

✅ Models loaded successfully on CUDA!


In [1]:
# ============================================================
#  VEHICLE DISTANCE MEASUREMENT SYSTEM  |  IP CAMERA MODE
#  (Original Author Logic - Adapted for Live Stream)
# ============================================================

import cv2
import numpy as np
from ultralytics import YOLO
import os
os.chdir('/home/bluro/Vehicle-Distance-Measurement-System')
# -------- ROI Zones (same as original, designed for 2042×1148px frame)
ROI_ZONES = {
    'LEFT': np.array([[240, 600], [925, 550], [312, 1100], [100, 1100]], dtype=np.int32),
    'MAIN': np.array([[925, 550], [1025, 550], [1712, 1100], [312, 1100]], dtype=np.int32),
    'RIGHT': np.array([[1025, 550], [1802, 600], [1942, 1100], [1712, 1100]], dtype=np.int32)
}

FOCAL_LENGTH = 500
OPTICAL_CENTERS = {'LEFT': (156, 1050), 'MAIN': (1000, 1020), 'RIGHT': (1868, 1050)}
TARGET_CLASSES = [2, 3, 5, 7]  # Car, Motorcycle, Bus, Truck
REAL_VEHICLE_HEIGHTS = {2: 1.55, 3: 1.2, 5: 3.0, 7: 2.5}
CONFIDENCE_THRESHOLD = 0.7
LICENSE_PLATE_CONFIDENCE = 0.475
MAX_DISPLAY_DISTANCE = 15
WARNING_DISTANCES = {'LEFT': 1, 'MAIN': 2, 'RIGHT': 1}
DISTANCE_DISPLAY_THRESHOLDS = {'LEFT': 5, 'RIGHT': 5}
MAIN_ROI_WARNING_THRESHOLD = 5.0

# -------- Load models (download from repo or same directory)
model = YOLO("yolov8n.pt").to(device)
license_plate_model = YOLO("license-plate.pt").to(device)

# -------- Utility Functions
def is_point_in_roi(point, roi_coordinates):
    return cv2.pointPolygonTest(roi_coordinates, point, False) >= 0

def get_vehicle_roi_zone(center_point):
    for zone_name, roi_coords in ROI_ZONES.items():
        if is_point_in_roi(center_point, roi_coords):
            return zone_name
    return None

def calculate_distance(bbox, class_id, zone_name):
    if class_id not in REAL_VEHICLE_HEIGHTS or zone_name not in OPTICAL_CENTERS:
        return 0
    x1, y1, x2, y2 = bbox
    bbox_height = y2 - y1
    if bbox_height <= 0:
        return 0
    vehicle_center_x, vehicle_center_y = (x1 + x2) / 2, (y1 + y2) / 2
    optical_center_x, optical_center_y = OPTICAL_CENTERS[zone_name]
    displacement = np.sqrt((vehicle_center_x - optical_center_x)**2 + (vehicle_center_y - optical_center_y)**2)
    distance = (REAL_VEHICLE_HEIGHTS[class_id] * FOCAL_LENGTH) / bbox_height
    return distance * (1.0 + displacement * 0.0001)

def should_display_distance(distance, zone_name):
    threshold = DISTANCE_DISPLAY_THRESHOLDS.get(zone_name, 999)
    return distance <= threshold

def get_distance_color(distance, zone_name):
    warning_distance = WARNING_DISTANCES.get(zone_name, 2.0)
    if zone_name == 'MAIN' and distance < MAIN_ROI_WARNING_THRESHOLD:
        return (0, 0, 255)
    elif distance < warning_distance:
        return (0, 0, 255)
    else:
        return (0, 255, 0)

def blur_license_plates_in_vehicle(frame, vehicle_bbox):
    x1, y1, x2, y2 = map(int, vehicle_bbox)
    vehicle_region = frame[y1:y2, x1:x2]
    if vehicle_region.size == 0:
        return
    try:
        plate_results = license_plate_model(vehicle_region, conf=LICENSE_PLATE_CONFIDENCE, verbose=False)
        for plate_result in plate_results:
            if plate_result.boxes is not None:
                for plate_box in plate_result.boxes.xyxy.cpu().numpy():
                    px1, py1, px2, py2 = map(int, plate_box)
                    frame_height, frame_width = frame.shape[:2]
                    px1, py1 = max(0, min(px1 + x1, frame_width)), max(0, min(py1 + y1, frame_height))
                    px2, py2 = max(0, min(px2 + x1, frame_width)), max(0, min(py2 + y1, frame_height))
                    if px2 > px1 and py2 > py1:
                        plate_region = frame[py1:py2, px1:px2]
                        if plate_region.size > 0:
                            blurred_plate = cv2.GaussianBlur(plate_region, (51, 51), 30)
                            frame[py1:py2, px1:px2] = blurred_plate
    except Exception as e:
        print(f"License plate detection error: {e}")

def draw_distance_label(frame, bbox, distance, color, zone_name):
    x1, y1, x2, y2 = map(int, bbox)
    distance_text = f"{distance:.1f}m"
    font = cv2.FONT_HERSHEY_DUPLEX
    font_scale, thickness = 1.2, 3
    (text_width, text_height), _ = cv2.getTextSize(distance_text, font, font_scale, thickness)
    frame_height, frame_width = frame.shape[:2]
    center_x = (x1 + x2) // 2
    text_x = max(10, min(center_x - text_width // 2, frame_width - text_width - 20))
    text_y = min(frame_height - 35, y2 + text_height + 20)
    padding = 10
    bg_x1, bg_y1 = max(0, text_x - padding), max(0, text_y - text_height - padding)
    bg_x2, bg_y2 = min(frame_width, text_x + text_width + padding), min(frame_height, text_y + padding)
    overlay = frame.copy()
    cv2.rectangle(overlay, (bg_x1, bg_y1), (bg_x2, bg_y2), color, -1)
    cv2.addWeighted(overlay, 0.8, frame, 0.2, 0, frame)
    cv2.putText(frame, distance_text, (text_x, text_y), font, font_scale, (255, 255, 255), thickness)

def draw_warning_message(frame, message):
    height, width = frame.shape[:2]
    font = cv2.FONT_HERSHEY_DUPLEX
    font_scale, thickness = 1.0, 3
    (text_width, text_height), _ = cv2.getTextSize(message, font, font_scale, thickness)
    position_x, position_y = width//2 - text_width//2, 80
    padding = 15
    overlay = frame.copy()
    cv2.rectangle(overlay, (position_x - padding, position_y - text_height - padding),
                  (position_x + text_width + padding, position_y + padding), (0, 0, 255), -1)
    cv2.addWeighted(overlay, 0.7, frame, 0.3, 0, frame)
    cv2.putText(frame, message, (position_x, position_y), font, font_scale, (255, 255, 255), thickness)

# -------- CAMERA STREAM (CHANGE URL)
camera_url = "http://192.168.110.186:8080/video"
video_capture = cv2.VideoCapture(camera_url)

if not video_capture.isOpened():
    raise RuntimeError("❌ Could not open IP camera stream!")

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("⚠️ Frame not received or stream lost.")
        break

    annotated_frame = frame.copy()
    results = model(annotated_frame, classes=TARGET_CLASSES, verbose=False)
    detections_info = []

    for result in results:
        if result.boxes is not None:
            for box in result.boxes:
                bbox = box.xyxy[0].cpu().numpy()
                x1, y1, x2, y2 = bbox
                center_point = (int((x1 + x2) / 2), int((y1 + y2) / 2))
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])
                class_name = model.names.get(class_id, "unknown")

                if class_name in ['car', 'motorcycle', 'bus', 'truck']:
                    blur_license_plates_in_vehicle(annotated_frame, bbox)

                zone_name = get_vehicle_roi_zone(center_point)
                if zone_name is None or confidence <= CONFIDENCE_THRESHOLD:
                    continue

                distance = calculate_distance(bbox, class_id, zone_name)
                warning_distance = WARNING_DISTANCES.get(zone_name, 2.0)

                if should_display_distance(distance, zone_name) and distance <= MAX_DISPLAY_DISTANCE:
                    color = get_distance_color(distance, zone_name)
                    draw_distance_label(annotated_frame, bbox, distance, color, zone_name)

                detections_info.append({'distance': distance, 'warning_distance': warning_distance})

    if detections_info and any(d['distance'] < d['warning_distance'] for d in detections_info):
        draw_warning_message(annotated_frame, "WARNING: VEHICLE VERY CLOSE!")

    cv2.imshow('Vehicle Distance Measurement (Live)', annotated_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("👋 Exiting...")
        break

video_capture.release()
cv2.destroyAllWindows()


NameError: name 'device' is not defined

In [4]:
import os
print(os.getcwd())

/home/bluro/Vehicle-Distance-Measurement-System


In [3]:
import os
os.chdir('/home/bluro/Vehicle-Distance-Measurement-System')
print("📂 Now working in:", os.getcwd())


📂 Now working in: /home/bluro/Vehicle-Distance-Measurement-System


In [5]:
from ultralytics import YOLO

model = YOLO("/home/bluro/Vehicle-Distance-Measurement-System/yolo12x.pt")
license_plate_model = YOLO("/home/bluro/Vehicle-Distance-Measurement-System/license-plate.pt")
